In [1]:
import matplotlib.pyplot as plt

import scipy.io
from sort import *
import numpy as np
import cv2
import pandas as pd
import ffmpeg
from tqdm.notebook import trange, tqdm
from collections import defaultdict
import json

In [19]:
a = ["1", "2", "3", "4", "5"]
"_".join(a[:-1])

'1_2_3_4'

In [2]:
pd_dataframe = pd.read_csv("au-currumbin4k.stream.20200608T000955960.detections.csv")

In [3]:
unique_times = np.sort(np.unique(pd_dataframe["timestamp"].to_numpy()))

In [4]:
pd_dataframe["frames_indx"] = 0
for i in range(len(unique_times)):
    pd_dataframe.loc[pd_dataframe['timestamp'] == unique_times[i], 'frames_indx'] = i

In [5]:
# Remove all non-surfers
pd_dataframe = pd_dataframe.drop(pd_dataframe[pd_dataframe["class"] < 5].index)

In [6]:
#create instance of SORT
mot_tracker = Sort(max_age=10, iou_threshold=0.1)

In [7]:
tracks_logger = {}
trajectory_logger = defaultdict(lambda: {"timestamps": [],
                                         "bounding_boxes": [], 
                                         "centroid": [], 
                                         "velocities": [],
                                         "number_of_detections": 0})

bb_x_size = 128
bb_y_size = 64

x_size = 3840
y_size = 2160

for i in range(pd_dataframe['frames_indx'].max() + 1):
    frame_detections = pd_dataframe[pd_dataframe['frames_indx'] == i]
    if len(frame_detections) > 0:
        centroid_x = (frame_detections["x0"] + (frame_detections["x1"] - frame_detections["x0"])/2).to_numpy()
        centroid_y = (frame_detections["y0"] + (frame_detections["y1"] - frame_detections["y0"])/2).to_numpy()
        
        centroid_x *= x_size
        centroid_y *= y_size

        score = frame_detections["confidence"].to_numpy()

        # Use centroid positions to create larger BB of points --> [x0,y0,x1,y1]
        detections = np.stack([centroid_x - bb_x_size//2, 
                               centroid_y - bb_y_size//2,
                               centroid_x + bb_x_size//2, 
                               centroid_y + bb_y_size//2,
                               score], 
                              1).astype(int)
    else:
        detections = np.empty((0, 5))

    trackers = mot_tracker.update(detections)

    tracks_logger[i] = trackers
    
    for track in trackers:
        timestamp = frame_detections["timestamp"].to_numpy()[0]
        bbox = track[:4].tolist()
        velocities = track[4:6].tolist()
        
        centroid = [bbox[0] + (bbox[2] - bbox[0])/2,
                   bbox[1] + (bbox[3] - bbox[1])/2]
        
        trajectory_logger[track[-1]]["timestamps"].append(timestamp)
        trajectory_logger[track[-1]]["bounding_boxes"].append(bbox)
        trajectory_logger[track[-1]]["centroid"].append(centroid)
        trajectory_logger[track[-1]]["velocities"].append(velocities)
        trajectory_logger[track[-1]]["number_of_detections"] += 1


In [8]:
with open('data.json', 'w') as fp:
    json.dump(trajectory_logger, fp)

In [9]:
with open('data.json', 'r') as fp:
    data = json.load(fp)

In [20]:
data[""]

{'9.0': {'timestamps': [1591574995.0,
   1591574995.24,
   1591574995.48,
   1591574995.76,
   1591574996.0,
   1591574996.24,
   1591574996.48,
   1591574996.76,
   1591574997.0,
   1591574997.24,
   1591574997.48,
   1591574997.76,
   1591574998.0],
  'bounding_boxes': [[3331.0, 1066.0, 3459.0, 1130.0],
   [3332.9998002397124, 1066.0, 3460.9998002397124, 1130.0],
   [3337.823305147109,
    1067.8823053112137,
    3465.823305147109,
    1131.8823053112137],
   [3340.3237191273515,
    1065.674622299043,
    3468.3237191273515,
    1129.674622299043],
   [3341.591495737208,
    1065.8449790792108,
    3469.591495737208,
    1129.8449790792108],
   [3343.3143369832337,
    1065.9120115300113,
    3471.3143369832337,
    1129.9120115300113],
   [3345.910053756602,
    1067.3702403477962,
    3473.910053756602,
    1131.3702403477962],
   [3348.7975270862316,
    1067.1830498670868,
    3476.7975270862316,
    1131.1830498670868],
   [3351.0960216976314,
    1067.1063063076144,
    3479.0

In [ ]:
for ids, trajectory in trajectory_logger.items():
    if trajectory["number_of_detections"] > 15:
        start_time = trajectory["timestamps"][0]
        end_time = trajectory["timestamps"][-1]
        ride_time = end_time - start_time
        
        centroid_start = trajectory["centroid"][0]
        centroid_end = trajectory["centroid"][-1]
        centroid_diff = np.subtract(centroid_end, centroid_start)
        
        dist = np.sqrt(centroid_diff[0]**2 + centroid_diff[0]**2)

        print("Ride %d, Time %.2f, Distance %.2f pixels" % (int(ids), ride_time, dist))


In [ ]:
start_centroid

In [ ]:
trajectory_logger[1572.0]['timestamps'][-1]

In [ ]:
total_tracking = np.concatenate(list(tracks_logger.values()))
unique_ids, counts = np.unique(total_tracking[:, -1], return_counts=True)
indx = np.argsort(counts)
sorted_counts = np.sort(counts)

In [ ]:
# Only count tracks with more than 30 steps
valid_tracks = unique_ids[indx[sorted_counts > 15]].tolist()

In [ ]:
valid_tracks

In [ ]:
indx = 0
points = np.stack(trajectory_logger[valid_tracks[indx]]['bounding_boxes'])
temp = trajectory_logger[valid_tracks[indx]]['timestamps']
velocities = np.stack(trajectory_logger[valid_tracks[indx]]['velocities'])

In [ ]:
vel_mag = np.sqrt(velocities[:, 0]**2 + velocities[:, 1]**2)

In [ ]:
print("Ride time %.2f seconds" % (temp[-1] - temp[0]))

In [ ]:
m_pix = 1.65/((pd_dataframe["y1"] - pd_dataframe["y0"]).mean() * y_size)

In [ ]:
(vel_mag * 3) * m_pix

In [ ]:
c = np.linspace(0, 1, len(points))
_ = plt.scatter(points[:, 0], points[:, 1], c=vel_mag)
plt.quiver(points[:, 0], points[:, 1], 
           velocities[:, 0], velocities[:, 1], 
           color='b', units='xy', width=10, scale=0.5)
plt.xlim([0, x_size])
plt.ylim([0, y_size])
_ = plt.title("Ride Trajectory")

In [ ]:
# Overlay onto video
vid = cv2.VideoCapture("au-currumbin4k.stream.20200608T000955960.bbox.mp4")
fps = vid.get(cv2.CAP_PROP_FPS)
timestamps = [vid.get(cv2.CAP_PROP_POS_MSEC)]
calc_timestamps = [0.0]

frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
size = (frame_width, frame_height)

save_vid = cv2.VideoWriter('test2.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         3, size)
    
tracks_mask = None
tracks_frame = None
valid_tracks_colours = np.random.randint(0, 255, (len(valid_tracks), 3))

for i in trange(pd_dataframe['frames_indx'].max(), leave=False):
    ret, frame = vid.read()

    if tracks_frame is None:
        tracks_mask = np.ones_like(frame)
        tracks_frame = np.zeros_like(frame)

    if i in tracks_logger.keys():
        tracks = tracks_logger[i]

        for track in tracks:
            if track[-1] in valid_tracks:
                center = np.array([track[0] + bb_x_size//2, track[1] + bb_y_size//2]).astype(int)
                track_indx = np.where(valid_tracks == track[-1])[0][0]
                color = valid_tracks_colours[track_indx].tolist()
                thickness = 10
                radius=30
                frame = cv2.circle(frame, center, radius, color, 3)
                
                arrow_start_point
                arrow_end_point
                frame = cv2.arrowedLine(frame, arrow_start_point, end_point,
                                     color, thickness) 

                start_point =  center + np.array([10, 15])
                end_point = center + np.array([-10, 15])
                tracks_mask = cv2.line(tracks_mask, start_point, end_point, (0, 0, 0), thickness) 
                tracks_frame = cv2.line(tracks_frame, start_point, end_point, color, thickness) 

    save_vid.write(frame * tracks_mask + tracks_frame)

vid.release()
save_vid.release()

In [ ]:
len(log_tracks)

In [ ]:
pd_dataframe['frames_indx'].max()

In [ ]:
tracks

In [ ]:
i

In [ ]:
len(log_tracks)